# Uncertainties and Randomization Demo

In [ ]:
from sorcha.modules.PPAddUncertainties import addUncertainties
from sorcha.modules.PPRandomizeMeasurements import randomizeAstrometry
from sorcha.modules.PPModuleRNG import PerModuleRNG

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord

This notebook demonstrates the uncertainties calculation and the randomisation of object position and magnitude based on the calculated uncertainties.

First we create the data. The expected LSST limiting magnitude for a single exposure in the g-band is 25 - we will extend our magnitude range to 30 for demonstrative purposes. The objects have identical position, velocity, seeing and limiting magnitude at the source position: uncertainties will thus depend on magnitude alone.

In [ ]:
obj_ids = np.arange(0, 1000)
obj_mags = np.linspace(15, 30, 1000)
psf_mags = obj_mags + 0.01
sig_limit = np.ones(len(obj_ids)) + 23.
seeing = np.ones(len(obj_ids))
astRArate = np.ones(len(obj_ids)) + 0.03
astDecrate = np.ones(len(obj_ids)) - 0.01
astRA = np.ones(len(obj_ids)) + 260.
astDec = np.ones(len(obj_ids)) -5.

In [ ]:
observations = pd.DataFrame({'ObjID': obj_ids,
                             'TrailedSourceMag': obj_mags,
                             'PSFMag': psf_mags,
                             'fiveSigmaDepthAtSource': sig_limit,
                             'seeingFwhmGeom': seeing,
                             'AstRARate(deg/day)': astRArate,
                             'AstDecRate(deg/day)': astDecrate,
                             'AstRA(deg)': astRA,
                             'AstDec(deg)': astDec})

In [ ]:
observations.columns

As can be seen from the columns, both TrailedSourceMag (the magnitude of the source including any trailing) and PSFMag (the magnitude of the object within the PSF-fitting kernel) are included here. The code will calculate the uncertainties for each one.

In [ ]:
configs = {'trailing_losses_on':True, 'default_SNR_cut': False}
rng = PerModuleRNG(2012)

In [ ]:
obs_uncert = addUncertainties(observations, configs, rng)

In [ ]:
obs_uncert.columns

In [ ]:
obs_uncert

As can be seen from the above, we have several new columns:

**observedTrailedSourceMag/observedPSFMag:** a randomised magnitude as 'observed' by the telescope, based on the 'true' magnitude and its calculated uncertainty

**PhotometricSigmaTrailedSource(mag)/PhotometricSigmaPSF(mag):** the uncertainty on the magnitude measurement

**AstrometricSigma(deg):** the uncertainty on the object position

**SNR:** the signal-to-noise ratio.

Let's take a look at these, starting with the SNR. For all plots, the single-exposure limiting magnitude in g-band (25) is marked.

In [ ]:
fig, ax = plt.subplots(1)
ax.plot(obs_uncert['TrailedSourceMag'].values, obs_uncert['SNR'].values, linestyle='', marker='.', color='rebeccapurple')
ax.axvline(25, color='black', linestyle='--')
ax.set_xlabel('magnitude')
ax.set_ylabel('SNR')

The photometric uncertainty for the trailed source and PSF magnitudes:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].plot(obs_uncert['TrailedSourceMag'].values, obs_uncert['PhotometricSigmaTrailedSource(mag)'].values, linestyle='', marker='.', color='thistle')
ax[0].set_xlabel('trailed source magnitude')
ax[0].set_ylabel('trailed source magnitude uncertainty (mag)')
ax[0].axvline(25, color='black', linestyle='--')
ax[1].plot(obs_uncert['PSFMag'].values, obs_uncert['PhotometricSigmaPSF(mag)'].values, linestyle='', marker='.', color='thistle')
ax[1].set_xlabel('PSF magnitude')
ax[1].set_ylabel('PSF magnitude uncertainty (mag)')
ax[1].axvline(25, color='black', linestyle='--')

Astrometric uncertainty:

In [ ]:
fig, ax = plt.subplots(1)
ax.plot(obs_uncert['TrailedSourceMag'].values, obs_uncert['AstrometricSigma(deg)'].values, linestyle='', marker='.', color='orchid')
ax.axvline(25, color='black', linestyle='--')
ax.set_xlabel('trailed source magnitude')
ax.set_ylabel('astrometric uncertainty (deg)')

Next we look at the randomisation of the magnitude based on the uncertainties, for both trailed source and PSF magnitudes. Note that for large magnitudes, and thus large photometric uncertainties, the randomisation leads to very large shifts in the magnitude. However, this only occurs for objects fainter than LSST's single exposure limiting magnitude - the SNR of these objects is so low that by default, SSPP will remove them from contention.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].plot(obs_uncert['TrailedSourceMag'].values, obs_uncert['observedTrailedSourceMag'].values, linestyle="", marker="x", color='mediumpurple')
ax[0].set_xlabel('trailed source magnitude before randomisation')
ax[0].set_ylabel('trailed source magnitude after randomisation')
ax[0].axvline(25, color='black', linestyle='--')
ax[1].plot(obs_uncert['PSFMag'].values, obs_uncert['observedPSFMag'].values, linestyle="", marker="x", color='mediumpurple')
ax[1].set_xlabel('trailed source magnitude before randomisation')
ax[1].set_ylabel('trailed source magnitude after randomisation')
ax[1].axvline(25, color='black', linestyle='--')

A separate function then randomises the object position based on its astrometric uncertainty. We will recreate the data to have the same magnitude, but vary RA and Dec.

In [ ]:
obj_ids = np.arange(0, 1000)
obj_mags = np.ones(len(obj_ids)) + 21.
psf_mags = obj_mags + 0.01
sig_limit = np.ones(len(obj_ids)) + 23.
seeing = np.ones(len(obj_ids))
astRArate = np.ones(len(obj_ids)) + 0.03
astDecrate = np.ones(len(obj_ids)) - 0.01
astRA = np.linspace(0, 360, 1000)
astDec = np.linspace(-90, 0, 1000)

In [ ]:
observations = pd.DataFrame({'ObjID': obj_ids,
                             'TrailedSourceMag': obj_mags,
                             'PSFMag': psf_mags,
                             'fiveSigmaDepthAtSource': sig_limit,
                             'seeingFwhmGeom': seeing,
                             'AstRARate(deg/day)': astRArate,
                             'AstDecRate(deg/day)': astDecrate,
                             'AstRA(deg)': astRA,
                             'AstDec(deg)': astDec})

In [ ]:
observations = addUncertainties(observations, configs, rng)

In [ ]:
observations['AstRATrue(deg)'] = observations['AstRA(deg)']
observations['AstDecTrue(deg)'] = observations['AstDec(deg)']
observations['AstRA(deg)'], observations['AstDec(deg)'] = randomizeAstrometry(observations, rng, sigName='AstrometricSigma(deg)', sigUnits='deg')

In [ ]:
true_coord = SkyCoord(ra=observations['AstRATrue(deg)'].values, dec=observations['AstDecTrue(deg)'].values, unit="deg")
random_coord = SkyCoord(ra=observations['AstRA(deg)'].values, dec=observations['AstDec(deg)'].values, unit="deg")

separation = true_coord.separation(random_coord).mas

In [ ]:
fig, ax = plt.subplots(1)
ax.hist(separation, 100, color='turquoise')
ax.set_ylabel('frequency')
ax.set_xlabel('difference between true and randomised position (mas)')